In [1]:
using MNIST, Distributions, TensorFlow

In [2]:
type DataLoader
    cur_id::Int
    order::Vector{Int}
end

DataLoader() = DataLoader(1, shuffle(1:60000))

function next_batch(loader::DataLoader, batch_size)
    x = zeros(Float32, batch_size, 784)
    y = zeros(Float32, batch_size, 10)
    for i in 1:batch_size
        x[i, :] = trainfeatures(loader.order[loader.cur_id])
        label = trainlabel(loader.order[loader.cur_id])
        y[i, Int(label)+1] = 1.0
        loader.cur_id += 1
        if loader.cur_id > 60000
            loader.cur_id = 1
        end
    end
    x, y
end

function load_test_set(N=10000)
    x = zeros(Float32, N, 784)
    y = zeros(Float32, N, 10)
    for i in 1:N
        x[i, :] = testfeatures(i)
        label = testlabel(i)
        y[i, Int(label)+1] = 1.0
    end
    x,y
end

load_test_set (generic function with 2 methods)

In [3]:
loader = DataLoader()
testx, testy = load_test_set()

(Float32[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], Float32[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0])

In [4]:
sess = Session()

2017-09-18 13:55:46.677836: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use SSE4.1 instructions, but these are available on your machine and could speed up CPU computations.
2017-09-18 13:55:46.677870: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use SSE4.2 instructions, but these are available on your machine and could speed up CPU computations.
2017-09-18 13:55:46.677876: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use AVX instructions, but these are available on your machine and could speed up CPU computations.
2017-09-18 13:55:47.860155: I tensorflow/core/common_runtime/gpu/gpu_device.cc:940] Found device 0 with properties: 
name: GeForce GTX 780
major: 3 minor: 5 memoryClockRate (GHz) 0.954
pciBusID 0000:03:00.0
Total memory: 2.95GiB
Free memory: 2.88GiB
2017-09-18 13:55:47.943317: W tensorflow/stream_executor/cuda/cuda_driver.cc:523]

Session(Ptr{Void} @0x00007eff837baa30)

In [5]:
x = placeholder(Float32)
y_ = placeholder(Float32)

<Tensor placeholder_2:1 shape=unknown dtype=Float32>

In [6]:
weight_variable(shape) = begin
    stddev = 0.1
    entry_num = prod(shape)
    initial = reshape(
        rand(Truncated(Normal(0, stddev), -2 * stddev, 2 * stddev), 
             entry_num), shape...) # this is not idential to truncated_normal
    Variable(map(x -> Float32(x), initial))
end

bias_variable(shape) = Variable(map(x -> Float32(x), 0.1 .* ones(shape)))

bias_variable (generic function with 1 method)

In [7]:
conv2d(x, W) = nn.conv2d(x, W, strides=[1, 1, 1, 1], padding="SAME")

max_pool_2x2(x) = nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding="SAME")

max_pool_2x2 (generic function with 1 method)

In [8]:
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

TensorFlow.Variables.Variable{Float32}(<Tensor node_2:1 shape=(1) dtype=Float32>, <Tensor node_2/Assign:1 shape=unknown dtype=Float32>)

In [9]:
x_image = reshape(x, [-1, 28, 28, 1])

<Tensor Reshape:1 shape=(?, 28, 28, 1) dtype=Float32>

In [10]:
h_conv1 = nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

<Tensor MaxPool:1 shape=(?, 14, 14, 32) dtype=Float32>

In [11]:
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

<Tensor MaxPool_2:1 shape=(?, 7, 7, 64) dtype=Float32>

In [12]:
W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = reshape(h_pool2, [-1, 7*7*64])
h_fc1 = nn.relu(h_pool2_flat * W_fc1 + b_fc1)

<Tensor Relu_3:1 shape=(?, 1024) dtype=Float32>

In [13]:
keep_prob = placeholder(Float32)
h_fc1_drop = nn.dropout(h_fc1, keep_prob)

<Tensor Dropout/Mul_2:1 shape=unknown dtype=Float32>

In [14]:
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv = h_fc1_drop * W_fc2 + b_fc2

<Tensor Add_6:1 shape=unknown dtype=Float32>

In [15]:
cross_entropy = reduce_mean(nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))

<Tensor reduce:1 shape=() dtype=Float32>

In [16]:
optimizer = train.AdamOptimizer(1e-4)
train_step = train.minimize(optimizer, cross_entropy)

<Tensor Group:1 shape=unknown dtype=Any>

In [17]:
correct_prediction = equal(indmax(y_conv, 2), indmax(y_, 2))
accuracy = reduce_mean(cast(correct_prediction, Float32))

<Tensor reduce_2:1 shape=unknown dtype=Float32>

In [18]:
run(sess, global_variables_initializer())

In [19]:
@time begin
    for i in 1:20000
        batch = next_batch(loader, 50)
        if i % 100 == 0
            train_accuracy = run(sess, accuracy, Dict(x=>batch[1], y_=>batch[2], keep_prob=>1.0))
            println(@sprintf("step %d, training accuracy %g", i, train_accuracy))
        end
        run(sess, train_step, Dict(x=>batch[1], y_=>batch[2], keep_prob=>0.5))
    
    end
    test_accuracy = run(sess, accuracy, Dict(x=>testx, y_=>testy, keep_prob=>1.0))
    println(@sprintf("test accuracy %g", test_accuracy))
end

step 100, training accuracy 0.88
step 200, training accuracy 0.86
step 300, training accuracy 0.94
step 400, training accuracy 0.9
step 500, training accuracy 0.9
step 600, training accuracy 0.92
step 700, training accuracy 0.92
step 800, training accuracy 0.9
step 900, training accuracy 0.94
step 1000, training accuracy 0.92
step 1100, training accuracy 0.98
step 1200, training accuracy 0.92
step 1300, training accuracy 0.98
step 1400, training accuracy 1
step 1500, training accuracy 0.98
step 1600, training accuracy 0.94
step 1700, training accuracy 0.98
step 1800, training accuracy 0.92
step 1900, training accuracy 0.9
step 2000, training accuracy 0.98
step 2100, training accuracy 0.96
step 2200, training accuracy 0.96
step 2300, training accuracy 0.98
step 2400, training accuracy 0.98
step 2500, training accuracy 0.98
step 2600, training accuracy 1
step 2700, training accuracy 0.98
step 2800, training accuracy 0.92
step 2900, training accuracy 1
step 3000, training accuracy 0.94
st

2017-09-18 14:00:57.224236: W tensorflow/core/common_runtime/bfc_allocator.cc:217] Allocator (GPU_0_bfc) ran out of memory trying to allocate 747.68MiB. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory is available.
2017-09-18 14:00:57.224289: W tensorflow/core/common_runtime/bfc_allocator.cc:217] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.59GiB. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory is available.
2017-09-18 14:00:57.224304: W tensorflow/core/common_runtime/bfc_allocator.cc:217] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.34GiB. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory is available.
2017-09-18 14:00:57.634744: W tensorflow/core/common_runtime/bfc_allocator.cc:217] Allocator (GPU_0_bfc) ran out of memory trying to allocate 3.90GiB. The cal